In [23]:
#! pip install -r requirements.txt
import pandas as pd, numpy as np
import glob, re,json,io,os
from datetime import datetime, timedelta
from time import sleep
import plotly.graph_objects as go
pd.options.display.max_colwidth = 100
from numpy.random import random
from numpy.random import choice
rng = np.random.default_rng()
from google.cloud import bigquery,storage
project_id = 'dett-stt'
clientB = bigquery.Client(project_id)
storageC=storage.Client(project_id)


In [24]:
#@title Lecture des issues et fermetures et génération des horodates de visites d'issues programmées
issues=pd.read_csv('https://exploitidf.github.io/IssuesTunnels/_downloads/caee2520ef1721f8437118adc2284ddf/issuesFermetures.csv')[[  'Tatouage',   'Fermeture']]
fermetures=issues['Fermeture'].value_counts()
lenFer=len(fermetures )
VisiteIssues=[]
for k in range(6):
  jours=pd.Series([(5+k*8)*7+ int(random()*4)*7+int(random()*4) for i in range(lenFer)],index=fermetures.index,name='jour')
  fermTmp=pd.concat([fermetures,jours],axis=1)
  isInd=issues.join(jours,on='Fermeture')
  isInd.index=[k*1000+j for j in issues.index]
  isInd['OT']=[345467+13000*k+m for m in issues.index]
  isInd['ordreVB']=k
  VisiteIssues.append(isInd)
VisiteIssues=pd.concat(VisiteIssues)
minutes=pd.Series([int(random()*240) for i in range(len(VisiteIssues))],index=VisiteIssues.index,name='minute')
VisiteIssues=pd.concat([VisiteIssues,minutes],axis=1)
VisiteIssues['HoroDate']=pd.to_datetime("24/"+VisiteIssues['jour'].astype(str)+" 23",format='%y/%j %H')+pd.to_timedelta(VisiteIssues['minute']*60000000000)
VisiteIssues=VisiteIssues[['Fermeture','ordreVB', 'OT','Tatouage', 'HoroDate']].reset_index(drop=True)


In [25]:
# Lecture du tableau des points de contrôle initial. Ce format devrait changer ...
vbIa=pd.read_csv('../_static/controleVB_IA.csv',encoding='UTF-8').iloc[:,1:]
lstPC=vbIa['PointControle'].value_counts().reset_index()
lstPC['ordrePC']=lstPC['PointControle'].str.split('.').apply(lambda x: x[0]).astype(int).astype(str).str.zfill(2) 
lstPC=lstPC.sort_values('ordrePC').reset_index(drop=True) #.set_index('ordrePC',drop=True)
lstLst=[]
for i in range(len(lstPC)) :
  vbIi=vbIa[vbIa['PointControle']==lstPC.loc[i,'PointControle']]
  lstCom=[]
  for j in vbIi.index:
    lstCom.append(vbIi.loc[j,'Note'][0]+'_'+vbIi.loc[j,'ResulControle'])
  lstLst.append(sorted(lstCom, reverse=False))

lstNbrResul=[len(ll) for ll in lstLst]
lsP=[[round(.2/(i-1),3)]*(i-1) +[.8]    for i in lstNbrResul]
lstLstR=[['R'+str(i) for i in range(j)] for j in lstNbrResul ]

tabRC=[]
for i in range(len(lstPC)) :
  for j in range(lstNbrResul[i]) :
    tabRC.append(['P'+ lstPC.loc[i,'ordrePC'], lstPC.loc[i,'PointControle'],lstLstR[i][j],lstLst[i][j],lsP[i][j]  ] )
pd.DataFrame(tabRC,columns=['codePC','PC','codeRC', 'RC', 'Probab']).to_csv('controleVB_IS.csv',index=False)
# Dans le futur, on devrait entrer directement une table de ce type.


In [53]:
ComAls=[x for x in  ['ComAl0_Très','ComAl1_îàèù','ComAl2_Trop Û','ComAl3_TroisÈ^cî','ComAl4_Qûàtré']]


def simuRapp(ferm,j):
    vst=VisiteIssues[(VisiteIssues['Fermeture']==ferm)&(VisiteIssues['ordreVB']==j)].reset_index(drop=True)  
    for k in range(len(vst)):
        dateFichier= str(int((vst.loc[k,'HoroDate']-datetime(2024,1,1)).total_seconds()+1000*random()))
        flNm='24-3/ISVB-'+ str(vst.loc[k,'OT'])+'-'+dateFichier+ '.json'
        flSr='{"Tatouage":" '+str(VisiteIssues.loc[k,'Tatouage'])+'",'
        flSr=flSr+'"HoroDate":"'+str(VisiteIssues.loc[k,'HoroDate'])+'",'
        flSr=flSr+'"Agent":"'+choice(['Karl','Karen', 'Kim','Kamel','Kun']) +'",'
        flSr=flSr+'"PC'+str(0)+'":"'+choice(a=lstLstR[0])+'"'+' ,"CM'+str(0)+'":"'+choice(a=ComAls)+'"'
        for n in range(1,len(lstPC)):
            flSr=flSr+' ,"PC'+str(n)+'":"'+choice(a=lstLstR[n])+'"'+' ,"CM'+str(n)+'":"'+choice(a=ComAls)+'"'            
        flSr=flSr+'}'
        with open(flNm,'bw') as fl:
            fl.write(flSr.encode('UTF8'))


In [54]:
# Création des rapports avec une probabilité variable par fermeture et par issue
if True :
    for ferm in list(fermetures.index):
        for j in range(6):
            if random()<0.1:
                simuRapp(ferm,j)

In [44]:

ComAls


[b'ComAl0_Tr\xc3\xa8s',
 b'ComAl1_\xc3\xae\xc3\xa0\xc3\xa8\xc3\xb9',
 b'ComAl2_Trop \xc3\x9b',
 b'ComAl3_Trois\xc3\x88^c\xc3\xae',
 b'ComAl4_Q\xc3\xbb\xc3\xa0tr\xc3\xa9']

In [67]:
bucket = storageC.get_bucket('issues-secours')
blobs = storageC.list_blobs(bucket)
for blob in blobs:
    if 'ISV' in blob.name:
        blob.delete()

filenames = next(os.walk('24-3'), (None, None, []))[2]
for fl in filenames[:10]:
    blob = bucket.blob('rapports-visites/'+fl)
    blob.upload_from_filename('24-3/'+ fl, if_generation_match= 0)
    sleep(3)





In [ ]:
# Liste des clés pour alimenter le schéma d'importation dans BQ
name = 'rapports-visites/' +filenames[5]
blob = bucket.blob(name)
fileContent= (blob.download_as_string(client=None).decode())
json.loads(fileContent).keys()

dict_keys(['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14'])

In [ ]:
# Test de l'importation dans BQ pour la fonction logDepot
client  = bigquery.Client()
dataset  = client.dataset('rapports_visites')
table = dataset.table('LogDepot')

def format_schema(schema):
        formatted_schema = []
        for row in schema:
            formatted_schema.append(bigquery.SchemaField(row,'STRING', 'NULLABLE'))
        return formatted_schema
lst_schema_VBIS = ['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 
                  'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14']


job_config = bigquery.LoadJobConfig()
job_config.schema = format_schema(lst_schema_VBIS)
stByt=','.join([flJson[k] for k in lst_schema_VBIS  ]).encode("utf-8")
job = client.load_table_from_file(io.BytesIO(stByt), table, job_config = job_config)